In [163]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from dotmap import DotMap
from pathlib import Path
from collections.abc import Mapping, Iterable

from IPython.display import display

%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [164]:
#with open("dfs_stock.pkl", 'rb') as f:
#    dfs_stock = pickle.load(f)
#with open("dfs_emissions.pkl", 'rb') as f:
#    dfs_emissions = pickle.load(f)
#with open("dfs_env_em_intensity.pkl", 'rb') as f:
#    dfs_env_em_intensity = pickle.load(f)
with open("dfs_prod_animals.pkl", 'rb') as f:
    dfs_prod_animals = pickle.load(f)
with open("trade_matrix.pkl", 'rb') as f:
    trade_matrix = pickle.load(f)
with open("trade_matrix_live_animals.pkl", 'rb') as f:
    trade_matrix_live_animals = pickle.load(f)

In [ ]:
def keep_non_redundant_info(trade_matrix,priority_reporter):
    """
        Replace columns ["Unit Importer Report","Quantity Importer Report","Unit Exporter Report","Quantity Exporter Report"]
        by columns ["Unit","Quantity"] removing redundance giving priority to info from priority_reporter (in {"importer","exporter"})
    """
    if priority_reporter == "importer":
        trade_matrix[["Unit","Quantity"]] = trade_matrix[["Unit Importer Report","Quantity Importer Report"]]
        trade_matrix["Unit"].fillna(trade_matrix["Unit Exporter Report"],inplace=True)
        trade_matrix["Quantity"].fillna(trade_matrix["Quantity Exporter Report"],inplace=True)
    else:
        trade_matrix[["Unit","Quantity"]] = trade_matrix[["Unit Exporter Report","Quantity Exporter Report"]]
        trade_matrix["Unit"].fillna(trade_matrix["Unit Importer Report"],inplace=True)
        trade_matrix["Quantity"].fillna(trade_matrix["Quantity Importer Report"],inplace=True)
    trade_matrix.drop(columns=["Unit Importer Report","Quantity Importer Report","Unit Exporter Report","Quantity Exporter Report"],inplace=True)

## Import Export Products

In [121]:
keep_non_redundant_info(trade_matrix,"importer")

In [125]:
trade_matrix["Equivalent Quantity (tonnes)"] = trade_matrix["Factor"] * trade_matrix["Quantity"]
detail_import = trade_matrix.groupby(by=["Item Group","Importer","Year","Exporter"]).sum()[["Equivalent Quantity (tonnes)"]]
total_export = detail_import.groupby(by=["Item Group","Year","Exporter"]).sum()

## Production Meat

## Import Export Live Animals

## Production Egg or Milk

In [152]:
print(dfs_prod_animals.keys())

odict_keys(['laying', 'production', 'producing_animals/slaughtered', 'milk_animals', 'prod_popultn', 'yield', 'yield/carcass_weight'])


In [128]:
print(dfs_prod_animals["laying"]["Item"].unique())
dfs_prod_animals["laying"].head()

['Eggs, hen, in shell' 'Eggs, other bird, in shell'
 'Eggs, hen, in shell (number)' 'Eggs Primary']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
0,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1961,Head,4000000.0,FAO estimate
1,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1962,Head,4400000.0,FAO estimate
2,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1963,Head,4600000.0,FAO estimate
3,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1964,Head,4800000.0,FAO estimate
4,2,Afghanistan,1062,"Eggs, hen, in shell",5313,1965,Head,5200000.0,FAO estimate


In [151]:
print(dfs_prod_animals["production"]["Item"].unique())
display(dfs_prod_animals["production"].head())
dfs_prod_animals["production"].groupby(by=["Unit","Item"]).sum()

['Eggs, hen, in shell' 'Eggs, hen, in shell (number)' 'Honey, natural'
 'Meat indigenous, camel' 'Meat indigenous, cattle'
 'Meat indigenous, chicken' 'Meat indigenous, goat'
 'Meat indigenous, sheep' 'Meat, cattle' 'Meat, game' 'Meat, goat'
 'Meat, sheep' 'Milk, whole fresh cow' 'Milk, whole fresh goat'
 'Milk, whole fresh sheep' 'Silk-worm cocoons, reelable' 'Wool, greasy'
 'Eggs, other bird, in shell' 'Eggs, other bird, in shell (number)'
 'Meat indigenous, pig' 'Meat, chicken' 'Meat, nes' 'Meat, pig'
 'Milk, whole fresh buffalo' 'Meat indigenous, horse'
 'Meat indigenous, rabbit' 'Meat indigenous, turkey' 'Meat, camel'
 'Meat, horse' 'Milk, whole fresh camel' 'Beeswax' 'Meat indigenous, duck'
 'Meat indigenous, geese' 'Skins, sheep, with wool' 'Hides, cattle, fresh'
 'Meat, duck' 'Meat, turkey' 'Meat, goose and guinea fowl' 'Meat, rabbit'
 'Skins, sheep, fresh' 'Meat indigenous, buffalo' 'Meat, buffalo'
 'Skins, goat, fresh' 'Meat indigenous, other camelids'
 'Meat indigenous, rode

,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
49,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1961,tonnes,10000.0,FAO estimate
50,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1962,tonnes,11000.0,FAO estimate
51,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1963,tonnes,11500.0,FAO estimate
52,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1964,tonnes,12000.0,FAO estimate
53,2,Afghanistan,1062,"Eggs, hen, in shell",5510,1965,tonnes,12800.0,FAO estimate


Area Code  Item Code  \
Unit   Item                                                        
Head   Meat indigenous, ass                     78254     534072   
       Meat indigenous, bird nes                61957     459616   
       Meat indigenous, buffalo                198485    1545480   
       Meat indigenous, camel                  274241    2120505   
       Meat indigenous, cattle                1295508    9704320   
       Meat indigenous, chicken               1305991   11275858   
       Meat indigenous, duck                   500290    4321730   
       Meat indigenous, geese                  261978    2230467   
       Meat indigenous, goat                  1143905    9370560   
       Meat indigenous, horse                  489096    4236960   
       Meat indigenous, mule                    31535     178716   
       Meat indigenous, other camelids          10017     123066   
       Meat indigenous, pig                   1177796    9869525   
       Meat indigenous, poultry               1305640   18293150   
       Meat indigenous, rabbit                 324267    3072784   
       Meat indigenous, rodents                 10017     122324   
       Meat indigenous, sheep                 1175604    9539112   
       Meat indigenous, turkey                 399583    3554490   
No     Eggs, hen, in shell                     658480    5450184   
       Eggs, hen, in shell (number)           1298496   10935683   
       Eggs, other bird, in shell              159485    1300472   
       Eggs, other bird, in shell (number)     300097    2418780   
tonnes Beef and Buffalo Meat                  1320177   18916044   
       Beeswax                                 365053    3418870   
       Eggs Primary                           1326646   18691189   
       Eggs, hen, in shell                    1326997   11134008   
       Eggs, other bird, in shell              312805    2584579   
       Hair, horse                               9958      88000   
       Hides, buffalo, fresh                   198485    1521630   
       Hides, cattle, fresh                   1291048    9425264   
...                                               ...        ...   
       Meat, cattle                           1320528    9081825   
       Meat, chicken                          1334457   11130160   
       Meat, duck                              511718    4411763   
       Meat, game                              433331    3630886   
       Meat, goat                             1164320    9406233   
       Meat, goose and guinea fowl             283936    2390644   
       Meat, horse                             512424    4311210   
       Meat, mule                               32130     179982   
       Meat, nes                               398722    3529482   
       Meat, other camelids                     10206     125064   
       Meat, other rodents                      10206     124308   
       Meat, pig                              1203413    9893565   
       Meat, rabbit                            346318    3223325   
       Meat, sheep                            1196458    9356729   
       Meat, turkey                            412711    3678480   
       Milk, whole fresh buffalo               133971    1177338   
       Milk, whole fresh camel                 205622    1550360   
       Milk, whole fresh cow                  1249664    8784720   
       Milk, whole fresh goat                  651810    5418240   
       Milk, whole fresh sheep                 490307    3700176   
       Milk,Total                             1269887   17919260   
       Offals, nes                              88082     731709   
       Sheep and Goat Meat                    1295893   18521750   
       Silk-worm cocoons, reelable             217998    1932735   
       Skins, furs                               2725      15535   
       Skins, goat, fresh                     1131220    9226025   
       Skins, sheep, fresh      

In [143]:
print(dfs_prod_animals["producing_animals/slaughtered"]["Item"].unique())
dfs_prod_animals["producing_animals/slaughtered"].head()

['Hides, cattle, fresh' 'Meat, camel' 'Meat, cattle' 'Meat, chicken'
 'Meat, goat' 'Meat, sheep' 'Skins, goat, fresh' 'Skins, sheep, fresh'
 'Meat, pig' 'Meat, horse' 'Meat, rabbit' 'Meat, turkey' 'Meat, duck'
 'Meat, goose and guinea fowl' 'Skins, sheep, with wool'
 'Hides, buffalo, fresh' 'Meat, buffalo' 'Meat, other camelids'
 'Meat, other rodents' 'Meat, ass' 'Meat, mule' 'Meat, bird nes'
 'Meat, game' 'Meat, nes' 'Beef and Buffalo Meat' 'Meat, Poultry'
 'Sheep and Goat Meat']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
145,2,Afghanistan,919,"Hides, cattle, fresh",5320,1961,Head,360000.0,FAO estimate
146,2,Afghanistan,919,"Hides, cattle, fresh",5320,1962,Head,384000.0,FAO estimate
147,2,Afghanistan,919,"Hides, cattle, fresh",5320,1963,Head,396000.0,FAO estimate
148,2,Afghanistan,919,"Hides, cattle, fresh",5320,1964,Head,402000.0,FAO estimate
149,2,Afghanistan,919,"Hides, cattle, fresh",5320,1965,Head,408000.0,FAO estimate


Area Code  Item Code  Element Code  \
Unit Item                                                              
Head Beef and Buffalo Meat          1324237   18961194      55854680   
     Hides, buffalo, fresh           198485    1521630       8458800   
     Hides, cattle, fresh           1291748    9431697      54599160   
     Meat, Poultry                  1335816   19034624      56019488   
     Meat, ass                        79758     538488       2585520   
     Meat, bird nes                   75613     628353       3070217   
     Meat, buffalo                   226561    1660091       9325960   
     Meat, camel                     280945    2146935      10134600   
     Meat, cattle                   1324588    9103500      55860000   
     Meat, chicken                  1336167   11139682      56024809   
     Meat, duck                      526714    4526146      22529114   
     Meat, game                         477      61639        281960   
     Meat, goat                     1181181    9503865      49715400   
     Meat, goose and guinea fowl     279852    2368111      11743447   
     Meat, horse                     507127    4307919      20891640   
     Meat, mule                       32130     179982        861840   
     Meat, nes                         3604      61798        281960   
     Meat, other camelids             10206     125064        574560   
     Meat, other rodents              10206     124308        574668   
     Meat, pig                      1211199    9947385      51130520   
     Meat, rabbit                    353989    3336284      15558604   
     Meat, sheep                    1203382    9414372      51263520   
     Meat, turkey                    422999    3748680      18469191   
     Sheep and Goat Meat            1306962   18626556      54838560   
     Skins, goat, fresh             1131676    9228075      47895960   
     Skins, sheep, fresh            1168271    9315190      49805840   
     Skins, sheep, with wool         281034    2250747      11985960   

                                      Year         Value  
Unit Item                                                 
Head Beef and Buffalo Meat        20874815  1.512987e+10  
     Hides, buffalo, fresh         3159330  1.247652e+09  
     Hides, cattle, fresh         20399793  1.450160e+10  
     Meat, Poultry                20932090  1.869302e+12  
     Meat, ass                      965925  1.222840e+08  
     Meat, bird nes                1147331  2.269256e+09  
     Meat, buffalo                 3484450  9.475568e+08  
     Meat, camel                   3788060  7.679914e+07  
     Meat, cattle                 20876805  1.419162e+10  
     Meat, chicken                20934080  1.715423e+12  
     Meat, duck                    8417906  1.039037e+11  
     Meat, game                     105311  0.000000e+00  
     Meat, goat                   18580005  1.551745e+10  
     Meat, goose and guinea fowl   4387872  2.651584e+10  
     Meat, horse                   7810560  2.396207e+08  
     Meat, mule                     321975  3.379057e+07  
     Meat, nes                      105311  0.000000e+00  
     Meat, other camelids           214650  2.985115e+07  
     Meat, other rodents            214650  4.174862e+09  
     Meat, pig                    19109489  6.569652e+10  
     Meat, rabbit                  5816545  4.088878e+10  
     Meat, sheep                  19159605  2.642682e+10  
     Meat, turkey                  6901150  2.390748e+10  
     Sheep and Goat Meat          20495565  4.185548e+10  
     Skins, goat, fresh           17893017  1.612604e+10  
     Skins, sheep, fresh          18609506  2.633272e+10  
     Skins, sheep, with wool       4481707  3.090197e+07

In [148]:
print(dfs_prod_animals["milk_animals"]["Item"].unique())
dfs_prod_animals["milk_animals"].head()

['Milk, whole fresh camel' 'Milk, whole fresh cow'
 'Milk, whole fresh goat' 'Milk, whole fresh sheep'
 'Milk, whole fresh buffalo' 'Milk,Total']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
703,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1961,Head,25000.0,FAO estimate
704,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1962,Head,28000.0,FAO estimate
705,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1963,Head,35000.0,FAO estimate
706,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1964,Head,33000.0,FAO estimate
707,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1965,Head,30000.0,FAO estimate


In [149]:
print(dfs_prod_animals["prod_popultn"]["Item"].unique())
dfs_prod_animals["prod_popultn"].head()

['Wool, greasy' 'Honey, natural' 'Beeswax' 'Silk-worm cocoons, reelable'
 'Hair, horse']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
1097,2,Afghanistan,987,"Wool, greasy",5319,1961,Head,18000000.0,FAO estimate
1098,2,Afghanistan,987,"Wool, greasy",5319,1962,Head,18150000.0,FAO estimate
1099,2,Afghanistan,987,"Wool, greasy",5319,1963,Head,18500000.0,FAO estimate
1100,2,Afghanistan,987,"Wool, greasy",5319,1964,Head,19000000.0,FAO estimate
1101,2,Afghanistan,987,"Wool, greasy",5319,1965,Head,19600000.0,FAO estimate


In [150]:
print(dfs_prod_animals["milk_animals"]["Item"].unique())
dfs_prod_animals["milk_animals"].head()

['Milk, whole fresh camel' 'Milk, whole fresh cow'
 'Milk, whole fresh goat' 'Milk, whole fresh sheep'
 'Milk, whole fresh buffalo' 'Milk,Total']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
703,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1961,Head,25000.0,FAO estimate
704,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1962,Head,28000.0,FAO estimate
705,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1963,Head,35000.0,FAO estimate
706,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1964,Head,33000.0,FAO estimate
707,2,Afghanistan,1130,"Milk, whole fresh camel",5318,1965,Head,30000.0,FAO estimate


In [147]:
print(dfs_prod_animals["yield"]["Item"].unique())
dfs_prod_animals["yield"].head()

['Eggs, hen, in shell' 'Hides, cattle, fresh' 'Milk, whole fresh camel'
 'Milk, whole fresh cow' 'Milk, whole fresh goat'
 'Milk, whole fresh sheep' 'Skins, goat, fresh' 'Skins, sheep, fresh'
 'Wool, greasy' 'Eggs Primary' 'Milk,Total' 'Milk, whole fresh buffalo'
 'Skins, sheep, with wool' 'Eggs, other bird, in shell'
 'Hides, buffalo, fresh' 'Honey, natural' 'Beeswax']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
258118,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1961,g/An,2500.0,Calculated data
258119,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1962,g/An,2500.0,Calculated data
258120,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1963,g/An,2500.0,Calculated data
258121,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1964,g/An,2500.0,Calculated data
258122,2,Afghanistan,1062,"Eggs, hen, in shell",5410,1965,g/An,2461.5,Calculated data


In [135]:
print(dfs_prod_animals["yield/carcass_weight"]["Item"].unique())
dfs_prod_animals["yield/carcass_weight"].head()

['Meat indigenous, camel' 'Meat indigenous, cattle'
 'Meat indigenous, chicken' 'Meat indigenous, goat'
 'Meat indigenous, sheep' 'Meat, camel' 'Meat, cattle' 'Meat, chicken'
 'Meat, goat' 'Meat, sheep' 'Beef and Buffalo Meat' 'Meat, Poultry'
 'Sheep and Goat Meat' 'Meat indigenous, pig' 'Meat, pig'
 'Meat indigenous, horse' 'Meat indigenous, rabbit'
 'Meat indigenous, turkey' 'Meat, horse' 'Meat, rabbit' 'Meat, turkey'
 'Meat indigenous, duck' 'Meat indigenous, geese' 'Meat, duck'
 'Meat, goose and guinea fowl' 'Meat indigenous, buffalo' 'Meat, buffalo'
 'Meat indigenous, other camelids' 'Meat indigenous, rodents'
 'Meat, other camelids' 'Meat, other rodents' 'Meat indigenous, ass'
 'Meat, ass' 'Meat indigenous, mule' 'Meat, bird nes' 'Meat, mule'
 'Meat indigenous, bird nes']


,Area Code,Area,Item Code,Item,Element Code,Year,Unit,Value,Flag
258278,2,Afghanistan,1137,"Meat indigenous, camel",5417,1961,kg/An,180.0,Calculated data
258279,2,Afghanistan,1137,"Meat indigenous, camel",5417,1962,kg/An,180.0,Calculated data
258280,2,Afghanistan,1137,"Meat indigenous, camel",5417,1963,kg/An,180.0,Calculated data
258281,2,Afghanistan,1137,"Meat indigenous, camel",5417,1964,kg/An,180.0,Calculated data
258282,2,Afghanistan,1137,"Meat indigenous, camel",5417,1965,kg/An,180.0,Calculated data


In [ ]:
# Harmonise units
## 1000 Head -> Head
unit_filter = trade_matrix_live_animals["Unit Importer Report"] == "1000 Head"
trade_matrix_live_animals.loc[unit_filter, "Quantity Importer Report"] = trade_matrix_live_animals[unit_filter]["Quantity Importer Report"].multiply(1000)
trade_matrix_live_animals.loc[unit_filter, "Unit Importer Report"] = "Head"

unit_filter = trade_matrix_live_animals["Unit Exporter Report"] == "1000 Head"
trade_matrix_live_animals.loc[unit_filter, "Quantity Exporter Report"] = trade_matrix_live_animals[unit_filter]["Quantity Exporter Report"].multiply(1000)
trade_matrix_live_animals.loc[unit_filter, "Unit Exporter Report"] = "Head"

## TODO 
## tonnes -> Head ???